In [1]:
import os
base_dir = os.getcwd()
train_dir = os.path.join(base_dir, 'dataset/train')
val_dir = os.path.join(base_dir, 'dataset/valid')
test_dir = os.path.join(base_dir, 'dataset/test')

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# Defining constants
img_size = (224, 224)
batch_size = 32
epochs = 20

# Loading dataset using ImageDataGenerator function
datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 2087 images belonging to 2 classes.
Found 334 images belonging to 2 classes.
Found 268 images belonging to 2 classes.


In [3]:
# Loading the MobileNet model with pre-trained weights, excluding the top layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Unfreezing some of the last layers of the base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Creating input tensor
inputs = Input(shape=(img_size[0], img_size[1], 3))

# Passing inputs through the base model
x = base_model(inputs, training=False)

# Adding custom layers on top of the base model
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

# Compiling the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

I0000 00:00:1722596955.454706    4541 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722596955.541931    4541 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722596955.542000    4541 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722596955.545334    4541 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722596955.545408    4541 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 7, 7, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,888,321 (14.83 MB)

 Trainable params: 3,864,897 (14.74 MB)

 Non-trainable params: 23,424 (91.50 KB)

In [5]:
# Defining callbacks
checkpoint_path = os.path.join(base_dir, 'sleep_detection_model.keras')
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

In [6]:
# Fine-tuning the model
history_fine = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

Epoch 1/20


/home/cosmos/miniconda3/envs/DataScienceGPU/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1722597119.396923    4682 service.cc:146] XLA service 0x7f83f8004620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722597119.397041    4682 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
I0000 00:00:1722597146.142216    4682 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 857ms/step - accuracy: 0.5452 - loss: 0.9786
Epoch 1: val_accuracy improved from -inf to 0.65569, saving model to /mnt/d/NullClass_Internship/Sleep_Detection/sleep_detection_model.keras
66/66 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.5455 - loss: 0.9776 - val_accuracy: 0.6557 - val_loss: 0.6147
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.6385 - loss: 0.7997
Epoch 2: val_accuracy improved from 0.65569 to 0.73952, saving model to /mnt/d/NullClass_Internship/Sleep_Detection/sleep_detection_model.keras
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 277ms/step - accuracy: 0.6385 - loss: 0.7997 - val_accuracy: 0.7395 - val_loss: 0.5446
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.6559 - loss: 0.7466
Epoch 3: val_accuracy improved from 0.73952 to 0.78443, saving model to /mnt/d/NullClass_Internship/Sleep_Detection/sleep_detection_model.keras
66/66 ━━━━━━━━━━━━━━━━━━━━ 20s 272ms/step - accuracy: 0.6560 - loss: 0.7464 - val_accuracy

In [3]:
import cv2
import numpy as np
from mtcnn import MTCNN
from tensorflow.keras.models import load_model
import gradio as gr
import tempfile
import os

base_dir = os.getcwd()
saved_model_dir = os.path.join(base_dir, 'saved_model')

# Loading the trained CNN model
model = load_model(saved_model_dir)

# Initializing the MTCNN face detector
detector = MTCNN()

In [16]:
# Making a function for fetching roi coordinates, performing classification and displaying image having detection
def classify_faces(img):
    faces = detector.detect_faces(img)
    sleepy_faces = 0

    for face in faces:
        x, y, w, h = face['box']
        x1 = face['keypoints']['left_eye'][0]
        y1 = face['keypoints']['left_eye'][1]
        x2 = face['keypoints']['right_eye'][0]
        y2 = face['keypoints']['right_eye'][1]

        # Calculating the distance between the eyes
        eye_distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

        if abs(x2 - x1) > abs(y2 - y1):
            # For larger horizontal distances between eyes
            roi_w = int(5 / 3 * eye_distance)
            roi_h = int(2 / 3 * eye_distance)
        else:
            # For larger vertical distances between eyes
            roi_w = int(2 / 3 * eye_distance)
            roi_h = int(5 / 3 * eye_distance)

        # Calculating the center point between the eyes
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Adjusting ROI coordinates to keep the center point between the eyes (It essentially grabs the top left
        # coordinate of the roi box)
        roi_x = int(center_x - roi_w / 2)
        roi_y = int(center_y - roi_h / 2)

        # Ensuring the ROI is within image boundaries
        roi_x = max(0, roi_x)
        roi_y = max(0, roi_y)
        roi_w = min(roi_w, img.shape[1] - roi_x)
        roi_h = min(roi_h, img.shape[0] - roi_y)

        crop = img[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]

        # Preprocessing the cropped face image
        crop_resized = cv2.resize(crop, (224, 224))
        crop_resized = crop_resized.astype('float32') / 255.0
        crop_resized = np.expand_dims(crop_resized, axis=0)

        prediction = model.predict(crop_resized)
        label = 'Awake' if prediction[0][0] < 0.5 else 'Sleepy'

        if label == 'Sleepy':
            sleepy_faces += 1
            # Drawing bounding box around drowsy face
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            # Putting text label above the bounding box
            cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Displaying the count of sleepy faces detected
    cv2.putText(img, f'Sleepy faces: {sleepy_faces}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return img, sleepy_faces

In [17]:
def process_image(image_path):
    img = cv2.imread(image_path)

    if img is None:
        raise ValueError(f"Unable to load image from {image_path}")

    # Converting BGR to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resizing the image to fit within a fixed window size while maintaining aspect ratio
    max_display_size = 800  # Maximum width or height for displaying the image
    height, width, _ = img_rgb.shape
    if max(height, width) > max_display_size:
        if height > width:
            new_height = max_display_size
            new_width = int(width * (max_display_size / height))
        else:
            new_width = max_display_size
            new_height = int(height * (max_display_size / width))
        img_rgb = cv2.resize(img_rgb, (new_width, new_height))

    # Classifying faces and retrieving image with bounding boxes
    img_with_boxes, sleepy_faces = classify_faces(img_rgb)

    # Converting back to BGR for saving with OpenCV
    img_with_boxes_bgr = cv2.cvtColor(img_with_boxes, cv2.COLOR_RGB2BGR)

    return img_with_boxes_bgr, f'Sleepy faces detected: {sleepy_faces}'

In [18]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    max_sleepy_faces = 0

    # Obtaining frame dimensions and FPS from the video capture
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Converting the frame from BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        frame_with_boxes, sleepy_faces = classify_faces(frame_rgb)
        frames.append(frame_with_boxes)

        # Updating maximum sleepy faces count if current frame has more
        if sleepy_faces > max_sleepy_faces:
            max_sleepy_faces = sleepy_faces

    cap.release()

    # Saving the processed video to a temporary file
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4')
    out = cv2.VideoWriter(temp_file.name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    for frame in frames:
        # Converting the frame back to BGR for saving
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        out.write(frame_bgr)

    out.release()

    return temp_file.name, f'The maximum number of sleepy faces detected in the entire video is: {max_sleepy_faces}'
